# Funciones

Temas:
1. Como declarar una función en Julia
2. "Duck-typing" en Julia
3. Funciones mutantes vs no mutantes en Julia
4. Algunas funciones de mayor orden

## Como declarar una función en Julia
Julia tiene varias formas de declarar funciones. La primera es utilizando las palabras clave `function` y `end`.

In [ ]:
function saludo(nombre)
    println("Hola $(nombre), ¡qué bueno verte!")
end

In [ ]:
function f(x)
    x^2
end
#Nota: podríamos haber escrito return x^2, y también habría funcionado
#Las funciones deJulia retornan el resultado de la última operación que
#hicieron.

Podemos llamar a cualquiera de estas funciones así:

In [ ]:
saludo("Mario")

In [ ]:
f(42)

Alternativamente podríamos haber declarado esas funciones en una sola línea

In [ ]:
salu2(nombre) = println("Hola $nombre, ¡qué bueno verte!")

In [ ]:
f2(x) = x^2

In [ ]:
salu2("María")

In [ ]:
f2(42)

Finalmente, podríamos haberlas declarado como funciones "anónimas"

In [ ]:
saludo3 = nombre -> println("Hola $nombre, ¡qué bueno verte!")

In [ ]:
f3 = x -> x^2

In [ ]:
saludo3("René")

In [ ]:
f3(42)

## Duck-typing in Julia
*"Si hace cuak como un pato, es un pato."* <br><br>
Las funciones de Julia van a funcionar siempre que el input tenga sentido. <br><br>
Por ejemplo, saludo funciona si el argumento es un número

In [ ]:
saludo(3.14)

Y `f` funcionará en una matriz. 

In [ ]:
A = rand(3, 3)
A

In [ ]:
f(A)

`f` también va a funcionar con un string, ya que `^` es el operador utilizado para repetir strings.

In [ ]:
f("hola")

Sin embargo, `f` no va a funcionar en un vector. A diferencia de `A^2`, que está bien definido, el significado de `v^2` donde `v` es un vector no es una operación algebráica bien definida.

In [ ]:
v = rand(3)

In [ ]:
# Esta línea dará error
f(v)

## Funciones mutantes vs. no-mutantes

A esta altura habrá notado que algunas funciones en los tutoriales anteriores tienen un signo `!`. Por convención, las funciones que terminan en un signo `!` pueden alterar el contenido de sus argumentos, y las que no lo tienen no.

Por ejemplo, miremos la diferencia entre las funciones de ordenamiento `sort` y `sort!`

In [ ]:
v = [3, 5, 2]

In [ ]:
sort(v)

In [ ]:
v

`sort(v)` retorna un arreglo ordenado de los elementos de `v`, pero `v` no cambia. <br><br>

Por otro lado, si usamos `sort!(v)`, los contenidos de `v` son ordenados dentro del mismo array.

In [ ]:
sort!(v)

In [ ]:
v

Cuando defina sus propias funciones trate de usar esta convención, de manera de saber rápido si la función que está utilizando corre el riesgo de modificar vectores que ha definido.

## Algunas funciones de mayor orden

### map

`map` es una función de "mayor orden" de Julia que *toma una función* como uno de sus argumentos. `map` luego aplica esa función a todos los elementos de una estructura de datos que le pases.
Por ejemplo, ejecutar

```julia
map(f, [1, 2, 3])
```
retornará un arreglo donde `f` fue aplicada a todos los elementos de `[1, 2, 3]`
```julia
[f(1), f(2), f(3)]
```

In [ ]:
map(f, [1, 2, 3])

Es decir, elevamos al cuadrado cada elemento de `[1, 2, 3]`.

También podríamos haberle dado a `map` una función anónima tal como

In [ ]:
x -> x^3

via

In [ ]:
map(x -> x^3, [1, 2, 3])

y ahora hemos elevado al cubo los elementos de `[1, 2, 3]`

### broadcast

`broadcast` es otra función de mayor orden como `map`. `broadcast` es una generaliación de `map`, por lo que puede hacer lo mismo que `map` y mucho más. La sintaxis de `broadcast` es la misma que para `map`

In [ ]:
broadcast(f, [1, 2, 3])

Y nuevamente aplicamos `f` a todos los elementos de `[1, 2, 3]`

Una abreviatura muy útil de `broadcast` es colocar `.` entre una función a la que quiera hacer `broadcast` y sus argumentos. Por ejemplo,

```julia
broadcast(f, [1, 2, 3])
```
es equivalente a
```julia
f.([1, 2, 3])
```

In [ ]:
f.([1, 2, 3])

Note nuevamente como esto es distinto a llamar
```julia
f([1, 2, 3])
```
Podemos elevar al cuadrado cada elemento del vector, ¡pero esto no es lo mismo que elevar al cuadrado el vector, ya que esto no está definido!

Para mostrar más claramente la diferencia, considere

```julia
f(A)
```
y
```julia
f.(A)
```
para una matriz `A`:

In [ ]:
A = [i + 3*j for j in 0:2, i in 1:3]

In [ ]:
f(A)

En este caso estamos haciendo
```
f(A) = A^2 = A * A
``` 

Por otro lado,

In [ ]:
B = f.(A)

contiene los cuadrados de cada una de las entradas individuales de `A`.

Esta sintaxis nos permite escribir expresiones complejas de una forma mucho más natural

In [ ]:
A .+ 2 .* f.(A) ./ A

en lugar de

In [ ]:
broadcast(x -> x + 2 * f(x) / x, A)

y ambas expresiones van a tener la misma performance.

**Pequeño paréntesis: macros**

Los macros nos permiten insertar de forma fácil expresiones en Julia, y comienzan con el símbolo `@`. La diferencia principal con una función es que los macros insertan expresiones en tiempo de compilación, y las funciones ejecutan expresiones en tiempo de cómputo. Si quiere aprender más sobre macros y expresiones puede leer la sección https://docs.julialang.org/en/v1/manual/metaprogramming/ de documentación de Julia.

Un macro que verá muchas veces en este curso es `@.`. Este macro modifica líneas de código y las convierte en broadcast. Así, al ejemplo anterior podríamos haberlo escrito de forma incluso más compacta como


In [ ]:
@. A + 2 * f(A) / A

### Ejercicios
#### 6.1 
Escriba una función `suma_uno` que sume 1 a su argumento

In [ ]:
function suma_uno(x)
    return x+1
end

In [ ]:
@assert suma_uno(1) == 2

In [ ]:
@assert suma_uno(11) == 12

#### 6.2 
Use `map` o `broadcast` para incrementar cada elemento de una matriz `A` en `1` y asígnelo a una variable `A1`.

In [ ]:
A1 = rand(3, 3)
A1 = map(x -> x+1, A)

In [ ]:
@assert A1 == [2 3 4; 5 6 7; 8 9 10]

#### 6.3 
Use la sintaxis `.` de `broadcast` para incrementar todos los valores de `A1` en `1` y guarde el resultado en `A2`

In [ ]:
A2 = rand(3, 3)
A2 .= A1 .+ 1

In [ ]:
@assert A2 == [3 4 5; 6 7 8; 9 10 11]

In [ ]:
# y ahora usando macro @.
A3 = rand(3, 3)
@. A3 = A + 1

In [ ]:
@assert A3 == [2 3 4; 5 6 7; 8 9 10]

#### 6.4

Cree una función `misuma(vec_1, vec_2)` que retorne la suma directa de `vec_1, vec_2`. Use para esto un loop `for` dentro de la función.
Luego cree otra función llamada `misuma!(vec_r, vec_1, vec_2)` que funcione igual, pero guarde el resultado en `vec_r`.

In [ ]:
#= 
function misuma(vec_1, vec_2)
    *código*
end
=#
function misuma(vec_1, vec_2)
    for i in 1:length(vec_1)
        return vec_1[i]+vec_2[i]
    end
end

In [ ]:
#= 
function misuma!(vec_r, vec_1, vec_2)
    *código*
end
=#
function misuma!(vec_r, vec_1, vec_2)
    for i in 1:length(vec_1)
        vec_r[i]=vec_1[i]+vec_2[i]
    end
end

In [ ]:
#Verificación de misuma. Si esta celda no da error, misuma funciona.
vec_1 = [1,2,3]
vec_2 = [3,2,1]

for elemento in misuma(vec_1, vec_2)
    @assert elemento == 4
end

In [ ]:
#Verificación de misuma!. Si esta celda no da error, misuma! funciona.
vec_1 = [1,2,3]
vec_2 = [3,2,1]
vec_r = copy(vec_1)
misuma!(vec_r, vec_1, vec_2)
for elemento in vec_r
    @assert elemento == 4
end

#### 6.5

Repita el ejercicio anterior, pero usando broadcast para escribir una función más compacta

In [ ]:
function misuma(vec_1, vec_2)
    return vec_1 .+ vec_2
end


function misuma!(vec_r,vec_1, vec_2)
    @. vec_r = vec_1 + vec_2
end

In [ ]:
vec_1 = [1,2,3]
vec_2 = [3,2,1]

for elemento in misuma(vec_1, vec_2)
    @assert elemento == 4
end

In [ ]:
vec_1 = [1,2,3]
vec_2 = [3,2,1]
vec_r = copy(vec_1)
misuma!(vec_r, vec_1, vec_2)
for elemento in vec_r
    @assert elemento == 4
end